In [17]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# Import the data
metric_data = pd.read_csv('/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/tirecsvfiles/B1320run50.csv', sep=',')
metric_data

# Iterate over columns and modify column names
for col in metric_data.columns:
    metric_data.rename(columns={col: f"{col} {metric_data[col][0]}"}, inplace=True)

# Drop the first row since it's now serving as part of the headers
metric_data = metric_data.drop(0)

# Reset index
metric_data.reset_index(drop=True, inplace=True)

# Convert all values to floats
metric_data = metric_data.apply(pd.to_numeric, errors='coerce')

# Drop any rows with NaN values (if necessary)
metric_data = metric_data.dropna()

# Reset index
metric_data.reset_index(drop=True, inplace=True)
metric_data.columns

/tmp/ipykernel_444/882446900.py:7: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.



Index(['ET sec', 'V kph', 'N rpm', 'SA deg', 'IA deg', 'RL cm', 'RE cm',
       'P kPa', 'FX N', 'FY N', 'FZ N', 'MX N-m', 'MZ N-m', 'NFX none',
       'NFY none', 'RST deg C', 'TSTI deg C', 'TSTC deg C', 'TSTO deg C',
       'AMBTMP deg F', 'SR none'],
      dtype='object')

In [34]:
Q1 = metric_data[['SA deg', 'FY N']].quantile(0.25)
Q3 = metric_data[['SA deg', 'FY N']].quantile(0.75)
IQR = Q3 - Q1

# Apply the IQR filter for each specified column
#metric_data = metric_data[~((metric_data[['SA deg', 'FY N']] < (Q1 - 1.5 * IQR)) | (metric_data[['SA deg', 'FY N']] > (Q3 + 1.5 * IQR))).any(axis=1)]





# Define the Pacejka Magic Formula for lateral force (FY) with five coefficients
def pacejka_model(alpha, B, C, D, E, F):
    return D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))) + F)

def fitted_curve(alpha, B_opt, C_opt, D_opt, E_opt, F_opt):
    return pacejka_model(alpha, B_opt, C_opt, D_opt, E_opt, F_opt)

# Define your bin edges based on the data range or desired intervals
pressure_bins = np.unique(np.round(metric_data['P kPa']))
camber_bins = np.unique(np.round(metric_data['IA deg']))

# Assuming load was varied in steps of 50 N
load_step = 50
min_load = int(metric_data['FZ N'].min() // load_step * load_step)
max_load = int(metric_data['FZ N'].max() // load_step * load_step)
load_bins = np.arange(min_load, max_load + load_step, load_step)

# Initialize empty lists to store binned data
binned_data = []

# Loop through each bin combination
for pressure in pressure_bins:
    for camber in camber_bins:
        for load in load_bins:
            # Filter data for current bin
            bin_filter = (np.round(metric_data['P kPa']) == pressure) & \
                         (np.round(metric_data['IA deg']) == camber) & \
                         (metric_data['FZ N'] >= load) & (metric_data['FZ N'] < load + load_step)
            filtered_data = metric_data[bin_filter]

            if not filtered_data.empty:
                # Calculate mean for 'SA deg' and 'FY N' in the current bin
                mean_SA = filtered_data['SA deg'].mean()
                mean_FY = filtered_data['FY N'].mean()

                # Store the mean values along with the bin identifiers
                binned_data.append([pressure, camber, load, mean_SA, mean_FY])

# Convert binned data into a DataFrame
binned_df = pd.DataFrame(binned_data, columns=['Pressure', 'Camber', 'Load', 'Mean SA', 'Mean FY'])


# Fit the Pacejka model to the binned data
initial_guess = [0.1, 0.1, 100, 0.1, 0]  # Update as needed

bounds = ([0, 0, 0, 0, 0],  # Lower bounds for B, C, D, E, F
          [np.inf, np.inf, np.inf, np.inf, np.inf])

# Ensure there's enough data to fit
if not binned_df.empty and len(binned_df) > len(initial_guess):
    params_optimal, _ = curve_fit(pacejka_model, binned_df['Mean SA'], binned_df['Mean FY'],  p0=initial_guess, maxfev=1500000)
    B_opt, C_opt, D_opt, E_opt, F_opt = params_optimal

    # Generate a range of slip angles (alpha) for plotting the fitted curve
    alpha_range = np.linspace(-10, 10, 500)

    # Calculate the fitted curve over the range of alpha
    fy_fitted = fitted_curve(alpha_range, *params_optimal)



In [35]:

import plotly.graph_objects as go

r2 = r2_score(binned_df['Mean FY'], pacejka_model(binned_df['Mean SA'], *params_optimal))

# Plot the original binned data and the fitted curve using Plotly
fig = go.Figure()

# Add binned original data
fig.add_trace(go.Scatter(x=binned_df['Mean SA'], y=binned_df['Mean FY'], mode='markers', name='Binned Original Data'))

# Add fitted curve
fig.add_trace(go.Scatter(x=alpha_range, y=fy_fitted, mode='lines', name='Fitted Curve'))

# Update plot layout with R² in the title
fig.update_layout(title=f'Pacejka Model Fit to Binned Data (R² = {r2:.3f})',
                  xaxis_title='Slip Angle (SA deg)',
                  yaxis_title='Lateral Force (FY N)',
                  legend_title='Data Type')

# Show plot
fig.show()

html_file_path = 'Pacejka_Model_Fit.html' # Change the file path as needed
fig.write_html(html_file_path)

# Print the optimized coefficients and the R² value
print("Optimized Coefficients:")
print("B:", B_opt)
print("C:", C_opt)
print("D:", D_opt)
print("E:", E_opt)
print("F:", F_opt)
print(f"R² Score: {r2:.3f}")



Optimized Coefficients:
B: -0.21721355145802385
C: 0.3259023430942629
D: 9103.432628022307
E: 0.5184172784269994
F: -2.836414744654072e-06
R² Score: 0.901


In [36]:
# After the binned_data list has been created

# Print out the unique camber and load bins used for fitting
print("Unique Camber Bins:", camber_bins)
print("Unique Load Bins:", load_bins)

# If you want to print each bin combination with its corresponding mean values
for entry in binned_data:
    pressure, camber, load, mean_SA, mean_FY = entry
    print(f"Pressure: {pressure} kPa, Camber: {camber} degrees, Load: {load} N - Mean SA: {mean_SA}, Mean FY: {mean_FY}")


Unique Camber Bins: [-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
Unique Load Bins: [-1650 -1600 -1550 -1500 -1450 -1400 -1350 -1300 -1250 -1200 -1150 -1100
 -1050 -1000  -950  -900  -850  -800  -750  -700  -650  -600  -550  -500
  -450  -400  -350  -300  -250  -200]
Pressure: 82.0 kPa, Camber: -5.0 degrees, Load: -1150 N - Mean SA: -7.131, Mean FY: 2704.38
Pressure: 82.0 kPa, Camber: -5.0 degrees, Load: -1100 N - Mean SA: -6.878, Mean FY: 2786.86
Pressure: 82.0 kPa, Camber: -4.0 degrees, Load: -1100 N - Mean SA: -5.734566666666666, Mean FY: 2601.928
Pressure: 82.0 kPa, Camber: -4.0 degrees, Load: -1050 N - Mean SA: -4.2955000000000005, Mean FY: 2357.02375
Pressure: 82.0 kPa, Camber: -3.0 degrees, Load: -1250 N - Mean SA: 3.8302, Mean FY: -2695.8
Pressure: 82.0 kPa, Camber: -3.0 degrees, Load: -1200 N - Mean SA: 4.827208333333334, Mean FY: -2875.925416666667
Pressure: 82.0 kPa, Camber: -2.0 degrees, Load: -1200 N - Mean SA: 5.984, Mean FY: -3065.74
Pressure: 82.0 kPa, Camber: -1.0 degr

In [16]:
# Print out the camber bin information
print("Unique camber angles used for binning (IA deg):")
print(camber_bins)

# Print out the load bin information
print("Unique vertical loads used for binning (FZ N):")
print(load_bins)


Unique camber angles used for binning (IA deg):
[-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
Unique vertical loads used for binning (FZ N):
[-1630. -1629. -1628. -1626. -1625. -1624. -1623. -1622. -1621. -1620.
 -1619. -1618. -1617. -1616. -1615. -1614. -1613. -1612. -1611. -1610.
 -1609. -1608. -1607. -1606. -1605. -1604. -1603. -1602. -1601. -1600.
 -1599. -1598. -1597. -1596. -1595. -1594. -1593. -1592. -1591. -1590.
 -1589. -1588. -1587. -1586. -1585. -1584. -1583. -1582. -1581. -1580.
 -1579. -1578. -1577. -1576. -1575. -1574. -1573. -1572. -1571. -1570.
 -1569. -1568. -1567. -1566. -1565. -1564. -1563. -1562. -1561. -1560.
 -1559. -1558. -1557. -1556. -1555. -1554. -1553. -1552. -1551. -1550.
 -1549. -1548. -1547. -1546. -1545. -1544. -1543. -1542. -1541. -1540.
 -1539. -1538. -1537. -1536. -1535. -1534. -1533. -1532. -1531. -1530.
 -1529. -1528. -1527. -1526. -1525. -1524. -1523. -1522. -1521. -1520.
 -1519. -1518. -1517. -1516. -1515. -1514. -1513. -1512. -1511. -1510.
 -1509.